<h1>Cleaning Student Loan Data from FAFSA</h1>
<br>Includes aggreagting the students and Total Amount of Loan per state
<br>Map the state abbrevations to 

In [ ]:
import pandas as pd

depression_data = pd.read_excel("data/college-loan/2012-13CampusBasedFunding.xlsx") 
state_names_df = pd.read_excel("Depression/StateNames.xlsx") 
